In [8]:
import cv2
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

## VGG16 with Algorithm


In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dense(4096, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.sparse_categorical_crossentropy)

In [ ]:
import tensorflow as tf
(Xtrain, Ytrain), (Xtest, Ytest) = tf.keras.datasets.mnist.load_data()
Xtrain = Xtrain[:, :, :, None] / 255.
Xtest = Xtest[:, :, :, None] / 255.
# Xtrain = tf.image.resize(Xtrain, (224, 224))
# Xtrain = tf.image.grayscale_to_rgb(Xtrain)
# Xtest = tf.image.resize(Xtest, (224, 224))
# Xtrain = tf.image.grayscale_to_rgb(Xtest)


In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator()
batch = 128
datagen.fit(Xtrain)
num_batch = len(Xtrain) / batch
Progbar = keras.utils.Progbar(num_batch)
for epoch in range(50):
    batches = 0
    for x, y in datagen.flow(Xtrain, Ytrain, batch_size=batch):
        x = tf.image.resize(x, (224, 224))
        x = tf.image.grayscale_to_rgb(x)
        history = model.fit(x, y, verbose=0)
        batches += 1
        Progbar.update(batches, values=[('loss', history.history['loss'][0])])
        if batches >= num_batch:
            print(epoch)
            break


## VGG16 with Keras

### option1

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
# vgg= VGG16(include_top=False, weights='imagenet')

In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, Model

vgg= VGG16(include_top=False, weights='imagenet')
#fit input 
x_in = layers.Input(shape=(32, 32, 1)) 
x = layers.Conv2D(3, 1)(x_in) # Conv2D <-(link_node) x_in
x = vgg(x)
#fit output
x = layers.Flatten()(x)
x = layers.Dense(10, activation="softmax")(x)
model = Model(x_in, x)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 3)         6         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten_8 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 14,719,824
Trainable params: 14,719,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Xtrain.shape

(60000, 28, 28, 1)

In [ ]:
Xtrain = tf.image.resize(Xtrain, (32, 32)) #vgg require 32*32

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')
model.fit(Xtrain, Ytrain, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.1246
Epoch 2/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0271
Epoch 3/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0186
Epoch 4/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0136
Epoch 5/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0108
Epoch 6/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0088
Epoch 7/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0074
Epoch 8/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0058
Epoch 9/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0048
Epoch 10/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0037


In [ ]:
import numpy as np
Xtest = tf.image.resize(Xtest, (32, 32))
Ztest = model.predict(Xtest)
np.sum(Ztest.argmax(axis=1) == Ytest)/len(Ztest)

0.9952

### option 2


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
vgg= VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3)) #Always 3D

#fit outpot
x = layers.Flatten()(vgg.output)
x = layers.Dense(10, activation="softmax")(x)
model = tf.keras.Model(vgg.input, x)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [ ]:
Xtrain = tf.image.grayscale_to_rgb(Xtrain)


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd')
model.fit(Xtrain, Ytrain, epochs=10)

In [ ]:
import numpy as np
Xtest = tf.image..grayscale_to_rgb(Xtest)
Ztest = model.predict(Xtest)
np.sum(Ztest.argmax(axis=1) == Ytest)/len(Ztest)

## Autoencoder

In [4]:
x = np.eye(8).astype(np.float32)
print(x)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [10]:
inputs = keras.Input(shape=x.shape[1])
encoder = keras.layers.Dense(3, activation='sigmoid')
decoder = keras.layers.Dense(x.shape[1], activation='sigmoid')
outputs = decoder(encoder(inputs))
model = Model(inputs, outputs)
model.compile(loss='mse', optimizer=keras.optimizers.SGD(learning_rate=3.5))

In [12]:
model.summary() #27 = (8*3) + 4(bias = node)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 27        
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 32        
Total params: 59
Trainable params: 59
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x, x, epochs=2000, verbose=0)

In [22]:
Z = model.predict(x)

In [20]:
np.round(Z)

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [24]:
  E = encoder(x)
  np.round(E)

array([[1., 0., 0.],
       [1., 1., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 1., 0.],
       [0., 1., 1.],
       [0., 0., 0.],
       [1., 0., 1.]], dtype=float32)

## Feature Extraction

In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

In [26]:
clf = SVC()
clf.fit(X, Y)
Z = clf.predict(Xtest)
print("Accuracy rate = ", accuracy_score(Ytest, Z))
print("Confusion Matrix:")
print(confusion_matrix(Ytest, Z))

NameError: ignored